In [ ]:
import os, shutil, json
import pandas as pd
import numpy as np
import scipy.io.wavfile as sc_wav

In [ ]:
MAIN_PATH = "gridspace-stanford-harper-valley/data"
SOUNDS_PATH = os.path.join(MAIN_PATH, "audio")

df = pd.read_csv("selected_samples.csv")
df.head(2)

In [ ]:
#shutil.rmtree('dataset')

In [ ]:
## 1. Copy all of the selected ones to the main path:
NEW_PATH = "dataset"
NEW_TRANSCRIPT_PATH = os.path.join(NEW_PATH, "transcript")
NEW_METADATA_PATH = os.path.join(NEW_PATH, "metadata")

NEW_SOUNDS_PATH = os.path.join(NEW_PATH, "audio")
AGENT_PATH = os.path.join(NEW_SOUNDS_PATH, 'agent')
CALLER_PATH = os.path.join(NEW_SOUNDS_PATH, 'caller')
MIXED_PATH = os.path.join(NEW_PATH, 'mixed')

for pth in [NEW_PATH, NEW_TRANSCRIPT_PATH, NEW_METADATA_PATH, NEW_SOUNDS_PATH, 
            AGENT_PATH, CALLER_PATH, MIXED_PATH]:
    if not os.path.exists(pth): os.mkdir(pth)

for idx, row in df.iterrows():
    sid = row['sid']
    # copy metadata:
    shutil.copy(row['metadata_path'], os.path.join(NEW_METADATA_PATH, f"{sid}.json"))
    #copy transcript:
    shutil.copy(row['transcript_path'], os.path.join(NEW_TRANSCRIPT_PATH, f"{sid}.json"))
    #copy audios 
    shutil.copy(os.path.join(SOUNDS_PATH, 'agent', f"{sid}.wav"), os.path.join(AGENT_PATH, f"{sid}.wav"))
    shutil.copy(os.path.join(SOUNDS_PATH, 'caller', f"{sid}.wav"), os.path.join(CALLER_PATH, f"{sid}.wav"))

In [ ]:
met_path = os.path.join(MAIN_PATH, 'metadata')
cc = 0
kk = 0
null_hg, caller_hg_first = 0, 0
for fn in os.listdir(met_path):
    meta_json = os.path.join(met_path, fn)
    with open(meta_json, 'r') as f:
        data = json.load(f)
    
    arrival_time_agent = data['agent']['arrival_time_ms']
    arrival_time_caller = data['caller']['arrival_time_ms']
    if arrival_time_agent < arrival_time_caller: cc+=1

    #hangup time
    hgup_time_agent = data['agent']['hangup_time_ms']
    hgup_time_caller = data['caller']['hangup_time_ms']
    if hgup_time_agent is None or hgup_time_caller is None:
        null_hg+=1
    else:
        if hgup_time_caller < hgup_time_agent: caller_hg_first+=1

    kk+=1
print(cc, kk)
print(caller_hg_first, null_hg)

In [ ]:
## 2. Created mixed audios
mixed_path = "mixed"

for idx, row in df.iterrows():
    call_id = row['sid']
    agent_wav = sc_wav.read(os.path.join(AGENT_PATH, f"{call_id}.wav"))
    caller_wav = sc_wav.read(os.path.join(CALLER_PATH, f"{call_id}.wav"))

    sampling_rate = agent_wav[0]

    ## Metadata:
    meta_json = os.path.join(NEW_METADATA_PATH, f"{call_id}.json")
    with open(meta_json, 'r') as f:
        data = json.load(f)
    start_time = data['start_time_ms']
    end_time = data['end_time_ms']
    call_duration_s = (end_time-start_time)/1000

    ## Mixed signal
    arr_size = int(sampling_rate * call_duration_s)
    mixed_arr = np.zeros(arr_size)

    ## agent arrives first always
    # caller can hangup before agent -> pad when that happens
    hgup_time_agent = data['agent']['hangup_time_ms']
    hgup_time_caller = data['caller']['hangup_time_ms']

    mixed = caller_wav[1].copy()

    if hgup_time_caller < hgup_time_agent:
        time_diff = hgup_time_agent - hgup_time_caller
        print(call_id)
    
    mixed[len(mixed)-len(agent_wav[1]):len(mixed)] += agent_wav[1]
    
    out_path = os.path.join(MIXED_PATH, f"{call_id}.wav")
    sc_wav.write(out_path, sampling_rate, mixed.astype(np.int16))


In [ ]:
df[['sid', 'code']]